In [209]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [210]:
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/"
annee="18"
numerosKalabas=[1,2,3,4,5]
numerosKalabas=[5]
nomKalabas=[repertoire+annee+"-K%d/"%num for num in numerosKalabas]

#nomKalabas=[home+"/Dropbox/Partage-Broadway/GAM/Caches/NosCaches/Kalabas/K2-Kalaba0304/"]

print "\n".join(nomKalabas)

lexiqueTestPrep=u"dans"
lexiqueTestNoun=u"loups"
lexiqueTestHyper=u"villagEOIs"

nomDeclarationRad="Declarations-Radicaux.tex"
nomDeclarationDec="Declarations-Decoupages.tex"
nomTableauxRad="Tableaux-Gloses.yaml"
nomDeclaration="Declarations.tex"
nomTableaux="Tableaux.yaml"


/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/18-K5/


### Javascript from :
https://stackoverflow.com/questions/12544056/how-to-i-get-the-current-ipython-notebook-name

In [211]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "theNotebook = " + "'"+thename+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [212]:
print(theNotebook)

Kalaba-Lexique-Series


In [213]:
debug=0
print_glose=True
print_cloze=True
print_radicaux=False
cloze_expanded=True
numeros={'1':'Un','2':'Deux','3':'Trois','4':'Quatre','5':'Cinq'}
personnes={'1sg':'UnSg','2sg':'DeuxSg','3sg':'TroisSg','1du':'UnDu','2du':'DeuxDu','3du':'TroisDu','1pl':'UnPl','2pl':'DeuxPl','3pl':'TroisPl'}

commandGrapho="\\newcommand{\\%s}{\\strutgb{\\graphoSkip}\\grapho{%s}}"
#commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}{\\dn %s}}"
#commandGrapho=u"\\newcommand{\\%s}{\\strutgb{0pt}{%s}}"
commandPhono=u"\\newcommand{\\%sP}{\\textipa{%s}}"
commandPhonoDecoupe=u"\\newcommand{\\%sD}{\\textipa{%s}}"
commandGlose=u"\\newcommand{\\%sG}{\\textGlose{%s}}"

In [214]:
def parse_grapho(graphie):
    def change_monograms(chaine):
        result=""
        for monogramme in chaine:
            if monogramme in monogrammes:
                result+=monogrammes[monogramme]
            else:
                result+=monogramme
        return result

    def change_bigrams(chaine):
        result=change_monograms(chaine)
        for bigramme in bigrammes:
            result=result.replace(bigramme,bigrammes[bigramme])
        return result

    
    chunks=re.findall(ur"\s+|[ptkqbdgmnNfTsSvDzZjwRrlLyvWJ]?[aeiouAEIUO]?", graphie,re.U)
    result=[]
    if debug: print graphie
    for chunk in chunks:
        if debug: print chunk,
        #ATTENTION : YAML interprète "no" comme FALSE
        if chunk in syllabes.keys():
            result.append(syllabes[chunk])
        else:
            result.append(chunk)
    if debug: print
    return change_bigrams("".join(result))

In [215]:
def parse_cloze(glose):
    if debug: print [glose]
    chunks=re.findall(r"\\cacherGloses{([^}]*)?}|(\w+)", glose,re.UNICODE)
    result=[]
    for chunk in chunks:
        result.extend([x for x in chunk if x!=""])
    return (u"%s;"%len(result)+";".join(result),"".join(result))

In [216]:
def recoder(chaine,table,boolUTF8=True):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    if boolUTF8:
        return result
    else:
        return result.encode("utf8")
#translit=string.maketrans(u'iueoaftgzZvjkSpN',u'tgazpHTGZJVkXyxI')

In [217]:
def escapeGabarit(forme):
    result=forme
    for i in "123":
        result=result.replace(i,"\\textnormal{C}$_{%s}$"%i)
        result=result.replace(str(int(i)+3),"\\textnormal{C}$_{%s}^{m}$"%i)
    result=result.replace("V","\\textnormal{V}")        
    result=result.replace("A","\\textnormal{V}$_{\\textnormal{\\tiny apo}}$")        
    return result

def escapeRacine(forme):
    m=re.match(ur"(.*)racine\(([^)]+)\)x(.*)",forme)
    if m:
        prefixe=m.group(1)
        print prefixe
        racine="\\racine{%s}"%m.group(2)
        print racine
        suffixe=m.group(3)
        suffixes=re.split(ur"([-+])",suffixe)
        gabarit="$\\times$\\gabarit{%s}"%escapeGabarit(suffixes[0])
        terminaison="".join(suffixes[1:])
        return prefixe+racine+gabarit+terminaison
    else:
        return forme

In [223]:
temp=u"mu-racine(releD)x1V2a3a-t-i"
escapeRacine(temp)
print case.radical

mu-
\racine{releD}
releD


In [219]:
for serie in nomKalabas:
    print "Génération lexique :",serie.split("/")[-2], time.strftime("%H:%M:%S")


    '''
    Lecture des fichiers de paramètres
     1) Phonology.yaml pour 
         - les voyelles, les consonnes
         - la correspondance pour l'écrit (symbole => lettre, syllabe => lettres)
         - la correspondance pour l'oral
         - la désaccentuation et les déligatures du français
         - les gabarits ?
         - l'apophonie, les mutations consonantiques
     2) Stems.yaml pour
         - les catégories et les classes
         - les radicaux et les formes correspondantes
             - la première forme sert de forme de citation
     3) MorphoSyntax.yaml pour
         - définition de la valeur des formes verbales du français
         - définition des attributs de chaque catégorie
         - valeur par défaut pour chaque attribut
         - définition des cas pour SUJ, OBJ, IND et PREP
         - .../...
         - définition des constituants et de l'ordre dans les syntagmes
         - définition des contractions du français (du => de le, etc.)
    '''
    with open(serie+"Phonology.yaml", 'r') as stream:
        phonology=yaml.load(stream)
    with open(serie+"Stems.yaml", 'r') as stream:
        stems=yaml.load(stream)
    with open(serie+"MorphoSyntax.yaml", 'r') as stream:
        morphosyntax=yaml.load(stream)
        
    '''
    Récupération des données du ParFuMor
    '''
    with open(serie+"Hierarchie-S2.pkl", 'rb') as input:
       PFM.hierarchieCF = pickle.load(input)
    with open(serie+"Lexique-S2.pkl", 'rb') as input:
       PFM.lexique = pickle.load(input)
    with open(serie+"Regles-S2.pkl", 'rb') as input:
       PFM.regles = pickle.load(input)

    if debug: print PFM.lexique.lexemes[lexiqueTestPrep]

    consonnes=phonology["consonnes"]
    voyelles=phonology["voyelles"]
    gabarits=phonology["gabarits"]
    derives=phonology["derives"]
    nom_classe=phonology["nom_classe"]
    nom_apo=phonology["apophonies"]
    nom_mut=phonology["mutations"]
    syllabes=phonology["syllabes"]

    if "Cas" in morphosyntax:
        casSyntagmes=morphosyntax["Cas"]
    else:
        casSyntagmes=""
    lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
    casPreposition={}
    for preposition in lexiquePrepositions:
        prep=preposition.upper()
        if casSyntagmes and prep in casSyntagmes:
            casPreposition[preposition]=casSyntagmes[prep]
        elif casSyntagmes and "PREP" in casSyntagmes:
            casPreposition[preposition]=casSyntagmes["PREP"]
        else:
            casPreposition[preposition]=""
    glosePREP={x:u"%s%s"%(x.upper(),cacherGloses("["+casPreposition[x].strip("+").capitalize()+"]")) for x in casPreposition if "+" in casPreposition[x]}

    try:
        __IPYTHON__ 
        ipython=True
    except: 
        ipython=False

    
    '''
    Constitution d'un entête pour les fichiers TEX en sortie
        - nom du script
        - date du run
    '''
    texHeader=[]
    version=theNotebook
    texHeader.append("%% script : "+version)
    texHeader.append('%%%% run : %s' % time.strftime("%y%m%d-%H%M"))
    if debug: print "\n".join(texHeader)
    
    if ipython or True:
    #    lexeme_nom="lexemes.txt"
    #    phrase_nom="phrases.txt"
        pass
    else:
        parser=optparse.OptionParser()
        parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
        parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
        parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
        parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

        (options, args) = parser.parse_args()
        lexeme_nom=args[0]
        phrase_nom=args[1]

    phonoIn =  unicode(phonology["translations"]["grapho"]["in"])
    graphoIn = [ord(char) for char in phonoIn]
    graphoOut = unicode(phonology["translations"]["grapho"]["out"])
    translit = dict(zip(graphoIn, graphoOut))
    if "monogrammes" in phonology["translations"]:
        monogrammes = phonology["translations"]["monogrammes"]
    else:
        monogrammes = {}
    bigrammes = phonology["translations"]["bigrammes"]

    accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
    deaccentIn = [ord(char) for char in accentedIn]
    deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
    deaccent = dict(zip(deaccentIn, deaccentOut))

    deligatures=phonology["translations"]["deligatures"]

    tipaIn = unicode(phonology["translations"]["ipa"]["in"])
    ipaIn = [ord(char) for char in tipaIn]
    ipaOut = unicode(phonology["translations"]["ipa"]["out"])
    toipa = dict(zip(ipaIn, ipaOut))

    tableaux={}
    tableauxGloses={}
    gloseClozes={}
    declarations=[]
    declarationsRad=[]
    declarationsDec=[]
    
    '''
    Pour chaque catégorie
    '''
    for categorie in PFM.lexique.catLexeme:
        '''
        si la catégorie n'est pas dans le tableau, il faut l'ajouter dans les trois structures :
            tableaux : les lignes à ajouter à la fin du document LaTeX sans les gloses
            tableauxGloses : les lignes à ajouter à la fin du document LaTeX avec les gloses
            gloseClozes : les informations sur les mots pour les questions CLOZE 
        '''
        if not categorie in tableaux:
            tableaux[categorie]=set()
            tableauxGloses[categorie]=set()
            gloseClozes[categorie]=[]
        if verbose: print categorie
        '''
        pour chaque lexème de la catégorie
        '''
        for lexeme in PFM.lexique.catLexeme[categorie]:
            if verbose: print PFM.lexique.lexemes[lexeme]
            '''
            pour chaque case du paradigme du lexème
            '''
            for case in PFM.lexique.lexemes[lexeme].paradigme.cases:
                caseGlose=case.glose
                '''
                Les éléments des catégories majeures (N,V,Adj) sont représentés en minuscules
                    => les noms propres commencent par une capitale
                Les éléments grammaticaux sont représentés en majuscules
                '''
                if categorie in PFM.categoriesMajeures:
                    nom=PFM.lexique.lexemes[lexeme].nom
                else:
                    nom=PFM.lexique.lexemes[lexeme].nom.upper()
                    gloseCase=case.glose
                    if categorie=="PREP" and gloseCase in glosePREP:
                        caseGlose=glosePREP[gloseCase]
                    elif categorie=="PREP":
                        caseGlose=caseGlose.upper()
                ref=PFM.modifierGlose(nom,case.sigma,"ref")
                ref=recoder(ref,deaccent)
                
                
                '''
                Dealing with LaTeX names format for commands
                
                 1) no ligatures
                 2) no numbers
                '''
                
                for ligature in deligatures:
                    ref=ref.replace(ligature,deligatures[ligature])
                for pers in personnes:
                    ref=ref.replace(pers,personnes[pers])
                for num in numeros:
                    ref=ref.replace(num,numeros[num])
                    
                grapho=chaine2utf8(re.sub(ur"\s+",ur"",parse_grapho(recoder(case.forme,translit))))
                phonoRad=case.detoure.replace(" ","")
                phonoDec=escapeRacine(case.decoupe)
                print phonoRad
                phono=case.forme.replace(" ","")

                '''
                Creating the DECLARATIONS TEX contents
                '''
                
                declarations.append(commandGrapho%(ref,grapho))
                declarations.append(commandPhono%(ref,phono))
                declarations.append(commandGlose%(ref,caseGlose))
                declarationsRad.append(commandGrapho%(ref,grapho))
                declarationsRad.append(commandPhono%(ref,phonoRad))
                declarationsRad.append(commandPhonoDecoupe%(ref,phonoDec))
                declarationsRad.append(commandGlose%(ref,caseGlose))
                declarationsDec.append(commandGrapho%(ref,grapho))
                declarationsDec.append(commandPhono%(ref,phonoDec))
                declarationsDec.append(commandGlose%(ref,caseGlose))
                
                '''
                Creating the TABLEAUX TEX contents
                '''
                
                tableauxGloses[categorie].add("\\"+ref+" & \\"+ref+"P & \\"+ref+"G \\\\")
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\blanc{\\"+ref+"G} \\\\")
                
                '''
                Creating the CLOZE CSV contents
                '''
                
                vedette=nom.split(".")[0]
                gloses,strGlose=parse_cloze(case.glose)
#                print gloses,case.decoupe,case.detoure
                phono=case.forme.replace(" ","")
                if debug: print [grapho, recoder(phono,toipa)]
                if cloze_expanded:
                    try:
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),recoder(case.decoupe,toipa),
                                         case.sigma,
                                         strGlose,
                                         gloses])
                    except NameError:
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),
                                         strGlose,
                                         gloses])
                else:
                    try:
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),recoder(case.decoupe,toipa),case.sigma,gloses])
                    except NameError:
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),gloses])
                gloseClozes[categorie].append(cloze)

        '''
        Écriture des fichiers en sortie
         1) déclarations sans les radicaux
         2) déclarations avec les radicaux
         3) données pour les questions CLOZE
         4) tableaux de vocabulaire avec gloses
         5) tableaux de vocabulaire sans gloses
        '''
        with codecs.open(serie+nomDeclaration, 'wb', encoding='utf8') as output:
            for declaration in texHeader+declarations:
        #        print type(declaration),declaration
                output.write(declaration+"\n")

        with codecs.open(serie+nomDeclarationRad, 'wb', encoding='utf8') as output:
            for declaration in texHeader+declarationsRad:
        #        print type(declaration),declaration
                output.write(declaration+"\n")

        with codecs.open(serie+nomDeclarationDec, 'wb', encoding='utf8') as output:
            for declaration in texHeader+declarationsDec:
        #        print type(declaration),declaration
                output.write(declaration+"\n")

        with codecs.open(serie+"Clozes.txt", 'wb', encoding='utf8') as output:
            for categorie in gloseClozes:
                output.write("#\t"+categorie+"\n#\n#\n")
                for cloze in gloseClozes[categorie]:
                    output.write(cloze+"\n")
                output.write("#\n#\n#\n")

        yaml.safe_dump(tableauxGloses, file(serie+nomTableauxRad, 'w'), encoding='utf-8', allow_unicode=True)
        yaml.safe_dump(tableaux, file(serie+nomTableaux, 'w'), encoding='utf-8', allow_unicode=True)

print u"Génération terminée :", time.strftime("%H:%M:%S")

Génération lexique : 18-K5 15:43:00

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}u

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}o

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}a

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}e

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}ut

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}ot

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}at

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}et

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}uT

\racine{Dazak}
\textRacineC{D}\textRacineV{a}\textRacineC{D}\textRacineC{z}a\textRacineC{k}oT

\racine{Dazak}
\t

\racine{Sinez}
\textRacineC{S}a\textRacineC{n}u\textRacineC{z}\textRacineC{z}ug

\racine{Sinez}
\textRacineC{S}a\textRacineC{n}u\textRacineC{z}\textRacineC{z}ag

\racine{Sinez}
\textRacineC{S}a\textRacineC{n}u\textRacineC{z}\textRacineC{z}eg

\racine{Sinez}
\textRacineC{S}a\textRacineC{n}u\textRacineC{z}\textRacineC{z}ig

\racine{Sinez}
\textRacineC{S}\textRacineV{i}\textRacineC{n}a\textRacineC{z}ub

\racine{Sinez}
\textRacineC{S}\textRacineV{i}\textRacineC{n}a\textRacineC{z}ab

\racine{Sinez}
\textRacineC{S}\textRacineV{i}\textRacineC{n}a\textRacineC{z}eb

\racine{Sinez}
\textRacineC{S}\textRacineV{i}\textRacineC{n}a\textRacineC{z}ib

\racine{Sinez}
\textRacineC{S}\textRacineV{i}\textRacineC{n}a\textRacineC{z}u

\racine{Sinez}
\textRacineC{S}\textRacineV{i}\textRacineC{n}a\textRacineC{z}a

\racine{Sinez}
\textRacineC{S}\textRacineV{i}\textRacineC{n}a\textRacineC{z}e

\racine{Sinez}
\textRacineC{S}\textRacineV{i}\textRacineC{n}a\textRacineC{z}i

\racine{Sinez}
\textRacineC{S}\textRacin


\racine{dezaf}
\textRacineC{d}o\textRacineC{z}u\textRacineC{f}\textRacineC{f}e

\racine{dezaf}
\textRacineC{d}o\textRacineC{z}u\textRacineC{f}\textRacineC{f}i

\racine{dezaf}
\textRacineC{d}o\textRacineC{z}u\textRacineC{f}\textRacineC{f}ug

\racine{dezaf}
\textRacineC{d}o\textRacineC{z}u\textRacineC{f}\textRacineC{f}ag

\racine{dezaf}
\textRacineC{d}o\textRacineC{z}u\textRacineC{f}\textRacineC{f}eg

\racine{dezaf}
\textRacineC{d}o\textRacineC{z}u\textRacineC{f}\textRacineC{f}ig

\racine{dezaf}
\textRacineC{d}\textRacineV{e}\textRacineC{z}a\textRacineC{f}ub

\racine{dezaf}
\textRacineC{d}\textRacineV{e}\textRacineC{z}a\textRacineC{f}ab

\racine{dezaf}
\textRacineC{d}\textRacineV{e}\textRacineC{z}a\textRacineC{f}eb

\racine{dezaf}
\textRacineC{d}\textRacineV{e}\textRacineC{z}a\textRacineC{f}ib

\racine{dezaf}
\textRacineC{d}\textRacineV{e}\textRacineC{z}a\textRacineC{f}u

\racine{dezaf}
\textRacineC{d}\textRacineV{e}\textRacineC{z}a\textRacineC{f}a

\racine{dezaf}
\textRacineC{d}\textRa

ka\textRadical{nunusa}tep
ka\textRadical{nunusa}te
ka\textRadical{nunusa}tef
ka\textRadical{nunusa}tes
ka-
\racine{nunusa}
ka\textRacineC{n}u\textRacineC{n}\textRacineC{n}a\textRacineC{s}up
ka-
\racine{nunusa}
ka\textRacineC{n}u\textRacineC{n}\textRacineC{n}a\textRacineC{s}u
ka-
\racine{nunusa}
ka\textRacineC{n}u\textRacineC{n}\textRacineC{n}a\textRacineC{s}uf
ka-
\racine{nunusa}
ka\textRacineC{n}u\textRacineC{n}\textRacineC{n}a\textRacineC{s}us
ka\textRadical{nunusa}dap
ka\textRadical{nunusa}da
ka\textRadical{nunusa}daf
ka\textRadical{nunusa}das
\textRadical{nunusa}teZep
\textRadical{nunusa}teZe
\textRadical{nunusa}teZef
\textRadical{nunusa}teZes

\racine{nunusa}
\textRacineC{n}u\textRacineC{n}\textRacineC{n}a\textRacineC{s}uZep

\racine{nunusa}
\textRacineC{n}u\textRacineC{n}\textRacineC{n}a\textRacineC{s}uZe

\racine{nunusa}
\textRacineC{n}u\textRacineC{n}\textRacineC{n}a\textRacineC{s}uZef

\racine{nunusa}
\textRacineC{n}u\textRacineC{n}\textRacineC{n}a\textRacineC{s}uZes
\textRadi

\racine{dugul}
\textRacineC{d}u\textRacineC{g}\textRacineC{g}a\textRacineC{l}uZe

\racine{dugul}
\textRacineC{d}u\textRacineC{g}\textRacineC{g}a\textRacineC{l}uZef

\racine{dugul}
\textRacineC{d}u\textRacineC{g}\textRacineC{g}a\textRacineC{l}uZes
\textRadical{dugul}daZep
\textRadical{dugul}daZe
\textRadical{dugul}daZef
\textRadical{dugul}daZes
lo\textRadical{dugul}tep
lo\textRadical{dugul}te
lo\textRadical{dugul}tef
lo\textRadical{dugul}tes
lo-
\racine{dugul}
lo\textRacineC{d}u\textRacineC{g}\textRacineC{g}a\textRacineC{l}up
lo-
\racine{dugul}
lo\textRacineC{d}u\textRacineC{g}\textRacineC{g}a\textRacineC{l}u
lo-
\racine{dugul}
lo\textRacineC{d}u\textRacineC{g}\textRacineC{g}a\textRacineC{l}uf
lo-
\racine{dugul}
lo\textRacineC{d}u\textRacineC{g}\textRacineC{g}a\textRacineC{l}us
lo\textRadical{dugul}dap
lo\textRadical{dugul}da
lo\textRadical{dugul}daf
lo\textRadical{dugul}das
\textRadical{dugul}tefip
\textRadical{dugul}tefi
\textRadical{dugul}tefif
\textRadical{dugul}tefis

\racine{dugul

\textRadical{ges}
\textRadical{lof}
\textRadical{jet}
\textRadical{Nuw}
\textRadical{zaS}
\textRadical{muk}
\textRadical{taje}
\textRadical{vom}
\textRadical{put}
\textRadical{seS}
\textRadical{kif}
\textRadical{nor}

\racine{Tibiw}
\textRacineC{T}\textRacineV{i}\textRacineC{b}\textRacineC{w}a\textRacineC{w}is
ke-
\racine{Tibiw}
ke\textRacineC{T}\textRacineV{i}\textRacineC{b}\textRacineC{w}a\textRacineC{w}i

\racine{Tibiw}
\textRacineC{T}\textRacineV{i}\textRacineC{b}\textRacineC{w}a\textRacineC{w}if

\racine{Tibiw}
\textRacineC{T}\textRacineV{i}\textRacineC{b}\textRacineC{w}a\textRacineC{w}ip
\textRadical{Tibiw}sas
ke\textRadical{Tibiw}sa
fo\textRadical{Tibiw}sa
\textRadical{Tibiw}sap
si-
\racine{Tibiw}
si\textRacineC{T}\textRacineV{i}\textRacineC{b}\textRacineC{b}a\textRacineC{w}o
ke-
\racine{Tibiw}
ke\textRacineC{T}\textRacineV{i}\textRacineC{b}\textRacineC{b}a\textRacineC{w}o
ve-
\racine{Tibiw}
ve\textRacineC{T}\textRacineV{i}\textRacineC{b}\textRacineC{b}a\textRacineC{w}o

\racine


\racine{veziN}
\textRacineC{v}o\textRacineC{z}\textRacineC{N}u\textRacineC{N}ifo
mu+
\racine{veziN}
mu\textRacineC{v}o\textRacineC{z}\textRacineC{N}u\textRacineC{N}iSo
ri-
\racine{veziN}
ri\textRacineC{v}\textRacineV{e}\textRacineC{z}a\textRacineC{N}a

\racine{veziN}
\textRacineC{v}\textRacineV{e}\textRacineC{z}a\textRacineC{N}ale

\racine{veziN}
\textRacineC{v}\textRacineV{e}\textRacineC{z}a\textRacineC{N}afo
mu+
\racine{veziN}
mu\textRacineC{v}\textRacineV{e}\textRacineC{z}a\textRacineC{N}aSo
si\textRadical{fibuN}se
\textRadical{fibuN}sele
\textRadical{fibuN}sefo
\textRadical{fibuN}seSu
ri-
\racine{fibuN}
ri\textRacineC{f}a\textRacineC{b}\textRacineC{N}u\textRacineC{N}i

\racine{fibuN}
\textRacineC{f}a\textRacineC{b}\textRacineC{N}u\textRacineC{N}ile

\racine{fibuN}
\textRacineC{f}a\textRacineC{b}\textRacineC{N}u\textRacineC{N}ifo
mu+
\racine{fibuN}
mu\textRacineC{f}a\textRacineC{b}\textRacineC{N}u\textRacineC{N}iSo
ri-
\racine{fibuN}
ri\textRacineC{f}\textRacineV{i}\textRacineC{b}a

Génération terminée : 15:43:07


In [220]:
bigrammes

{'D': '^s', 'J': '`', 'N': '.s', 'T': '_t', 'W': "'", 'Z': '_h'}

In [221]:
[l for l in declarations if u"être" in l]

[u'\\newcommand{\\etreVtPrsTroisSgAG}{\\textGlose{\xeatre\\cacherGloses{.VT.V2}\\cacherGloses{xV2.PRS}\\cacherGloses{-V2.A}\\cacherGloses{-V2.3Sg}}}',
 u'\\newcommand{\\etreVtPrsTroisSgBG}{\\textGlose{\xeatre\\cacherGloses{.VT.V2}\\cacherGloses{xV2.PRS}\\cacherGloses{-V2.B}\\cacherGloses{-V2.3Sg}}}',
 u'\\newcommand{\\etreVtPrsTroisSgCG}{\\textGlose{\xeatre\\cacherGloses{.VT.V2}\\cacherGloses{xV2.PRS}\\cacherGloses{-V2.C}\\cacherGloses{-V2.3Sg}}}',
 u'\\newcommand{\\etreVtPrsTroisSgDG}{\\textGlose{\xeatre\\cacherGloses{.VT.V2}\\cacherGloses{xV2.PRS}\\cacherGloses{-V2.D}\\cacherGloses{-V2.3Sg}}}',
 u'\\newcommand{\\etreVtPrsTroisDuAG}{\\textGlose{\xeatre\\cacherGloses{.VT.V2}\\cacherGloses{xV2.PRS}\\cacherGloses{-V2.A}}}',
 u'\\newcommand{\\etreVtPrsTroisDuBG}{\\textGlose{\xeatre\\cacherGloses{.VT.V2}\\cacherGloses{xV2.PRS}\\cacherGloses{-V2.B}}}',
 u'\\newcommand{\\etreVtPrsTroisDuCG}{\\textGlose{\xeatre\\cacherGloses{.VT.V2}\\cacherGloses{xV2.PRS}\\cacherGloses{-V2.C}}}',
 u'\\newcomm

In [222]:
chaine="tajewu"
print "parse", parse_grapho(chaine)
print "parse, recoder",recoder(parse_grapho(chaine),translit)
print "recoder", recoder(chaine,translit)
print "recoder, parse",parse_grapho(recoder(chaine,translit))


parse tajewu
parse, recoder biJuWe
recoder biJuWe
recoder, parse bi`u'e
